In [1]:
import torch

from torch_geometric.utils import to_dense_adj
from torch_geometric.nn.pool import global_mean_pool
from torch.utils.data import DataLoader, random_split

In [2]:
from helpers import CVFConfigDataset

In [3]:
# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )
# dataset = CVFConfigDataset(
#     "graph_1_config_rank_dataset.csv", "graph_1_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_3_config_rank_dataset.csv", "graph_3_edge_index.json", 6
# )
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
# dataset = CVFConfigDataset(
#     "graph_5_config_rank_dataset.csv", "graph_5_edge_index.json", 9
# )
# dataset = CVFConfigDataset(
#     "graph_6_config_rank_dataset.csv", "graph_6_edge_index.json", 5
# )
# dataset = CVFConfigDataset(
#     "graph_6b_config_rank_dataset.csv", "graph_6b_edge_index.json", 5
# )
# dataset = CVFConfigDataset(
#     "graph_7_config_rank_dataset.csv", "graph_7_edge_index.json", 6
# )
# dataset = CVFConfigDataset(
#     "graph_8_config_rank_dataset.csv", "graph_8_edge_index.json", 7
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n5_config_rank_dataset.csv", "graph_powerlaw_cluster_graph_n5_edge_index.json", 5
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n6_config_rank_dataset.csv", "graph_powerlaw_cluster_graph_n6_edge_index.json", 6
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv", "graph_powerlaw_cluster_graph_n7_edge_index.json", 6
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n8_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n8_edge_index.json",
#     8
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n9_edge_index.json",
#     8
# )
# dataset = CVFConfigDataset(
#     "graph_powerlaw_cluster_graph_n12_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n12_edge_index.json",
#     10
# )

dataset = CVFConfigDataset(
    "implicit_graph_n10_config_rank_dataset.csv",
    "implicit_graph_n10_edge_index.json",
    3,
    one_hot_encode=False
)

train_split = 0.8
dev_split = 0.1
test_split = 0.1


train_set, dev_set, test_set = random_split(dataset, [train_split, dev_split, test_split])

train_loader = DataLoader(train_set, batch_size=1024, shuffle=False)
dev_loader = DataLoader(dev_set, batch_size=1024, shuffle=False)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
adjacency = to_dense_adj(dataset.edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency = adjacency.unsqueeze(0).to(device)
adjacency

tensor([[[1., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 1., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 1., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 1., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 1., 1.]]], device='cuda:0')

In [6]:
def add_graph_properties(x):
    """ this needs to be optimized """
    inp = torch.tensor([]).to(device)
    for each in x:
        inp = torch.cat((inp, torch.mul(each, adjacency)))
    return inp

In [22]:
class MLPNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.layer1 = torch.nn.Linear(dim_in, dim_h)
        self.layer2 = torch.nn.Linear(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x):
        h = self.layer1(x)
        h = torch.relu(h)
        h = self.layer2(h)
        h = torch.relu(h)
        h = self.out(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, train_loader, epochs):
        # criterion = torch.nn.CrossEntropyLoss()
        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        for epoch in range(1, epochs + 1):
            self.train()
            total_loss = 0
            count = 0
            for batch in train_loader:
                x = batch[0].to(device)
                x = add_graph_properties(x)
                y = batch[1].to(device)
                y = y.unsqueeze(0).reshape(-1, 1, 1).float()
                optimizer.zero_grad()
                out = self(x)
                #print("output", out.shape, "y", y.shape)
                loss = criterion(out, y)
                total_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Epoch:", epoch, "Loss:", total_loss / count)


In [23]:
print("number of nodes", dataset.nodes)
mlp = MLPNN(dataset.nodes, 32, 1).to(device)
print(mlp)

mlp.fit(train_loader, epochs=20)

number of nodes 10
MLPNN(
  (layer1): Linear(in_features=10, out_features=32, bias=True)
  (layer2): Linear(in_features=32, out_features=32, bias=True)
  (out): Linear(in_features=32, out_features=1, bias=True)
)
Epoch: 1 Loss: tensor(2329.1191, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 2 Loss: tensor(542.7955, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 3 Loss: tensor(492.3096, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 4 Loss: tensor(459.9273, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 5 Loss: tensor(430.3346, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 6 Loss: tensor(409.0277, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 7 Loss: tensor(398.6284, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 8 Loss: tensor(393.0577, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 9 Loss: tensor(387.6041, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 10 Loss: tensor(381.1038, device='cuda:0', grad_fn=<DivBackward0>)
Epoch: 11 Loss: tensor(373.9460, device='cuda:0', grad_

In [26]:
# testing
torch.no_grad()
torch.set_printoptions(profile="full")

total_matched = 0
edge_index = dataset.edge_index.t().to(device)

for batch in test_loader:
    x = batch[0].to(device)
    x = add_graph_properties(x)
    y = batch[1].to(device)
    y = y.unsqueeze(0).reshape(-1, 1, 1).float()
    predicted = mlp(x)
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched

print("Total matched", total_matched, "out of", len(test_set), "| Accuracy", round(total_matched/len(test_set) * 100, 4), "%")

Total matched 156 out of 5904 | Accuracy 2.6423 %
